In [ ]:
import pandas as pd
from pathlib import Path
import openpyxl as oxl
import numpy as np
import pkg_resources
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

df = pd.read_excel(Path("data") / "credit.xlsx")
df.info()

df["Cible"] = df["Cible"].map({1: 0, 2: 1})
df["Cible"].value_counts()

ProfileReport(df, title="Rapport de données - Crédit")

var_num = df[["Age", "Montant_credit", "Duree_credit"]]
var_cat = df.drop(columns=["Cle", "Age", "Montant_credit", "Duree_credit"])

print("Statistiques descriptives variables numériques :")
var_num.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Cle                  1000 non-null   int64 
 1   Comptes              1000 non-null   int64 
 2   Duree_credit         1000 non-null   int64 
 3   Historique_credit    1000 non-null   object
 4   Objet_credit         1000 non-null   object
 5   Montant_credit       1000 non-null   int64 
 6   Epargne              1000 non-null   int64 
 7   Anciennete_emploi    1000 non-null   int64 
 8   Taux_effort          1000 non-null   int64 
 9   Situation_familiale  1000 non-null   object
 10  Garanties            1000 non-null   object
 11  Anciennete_domicile  1000 non-null   int64 
 12  Biens                1000 non-null   object
 13  Age                  1000 non-null   int64 
 14  Autres_credits       1000 non-null   object
 15  Statut_domicile      1000 non-null   object
 16  Nb_cred

,Age,Montant_credit,Duree_credit
count,1000.000000,1000.000000,1000.000000
mean,35.546000,3271.258000,20.903000
std,11.375469,2822.736876,12.058814
min,19.000000,250.000000,4.000000
25%,27.000000,1365.500000,12.000000
50%,33.000000,2319.500000,18.000000
75%,42.000000,3972.250000,24.000000
max,75.000000,18424.000000,72.000000


In [7]:
# Import des librairies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm

In [ ]:
# encodage de toutes les variables
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
df = pd.DataFrame({
'Name' : ['Sweet Mask', 'Bald Cape', 'Blizzard of Hell',
'King', 'Glasses', 'Metal Bat', 'Mumen Rider'],
'Class' : ['A', 'C', 'B', 'S', 'A', 'S', 'C']})
le = LabelEncoder()
oe = OrdinalEncoder(categories=[['S', 'A', 'B', 'C']])
df['label'] = le.fit_transform(df[['Class']])
df['ord'] = oe.fit_transform(df[['Class']])

In [21]:
# Séparation train/test
X = df.drop(columns=['Cible'])
y = df['Cible']

# 80% train / 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# estimation d'un modèle logistique

In [ ]:
# --- 1. Modèle Logistique ---
X_train_sm = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_sm).fit(disp=0)
X_test_sm = sm.add_constant(X_test, has_constant='add')
y_pred_logit = logit_model.predict(X_test_sm)
auc_logit = roc_auc_score(y_test, y_pred_logit)
print(f"Logit ROC AUC : {auc_logit:.3f}")

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

# estimation d'un modèle probit

In [ ]:
# --- 2. Modèle Probit ---
probit_model = sm.Probit(y_train, X_train_sm).fit(disp=0)
y_pred_probit = probit_model.predict(X_test_sm)
auc_probit = roc_auc_score(y_test, y_pred_probit)
print(f"Probit ROC AUC : {auc_probit:.3f}")

# estimation d'un modèle Catboost

In [28]:
# --- 3. CatBoost ---
cat_model = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=6, verbose=0)
cat_model.fit(X_train, y_train)
y_pred_catboost = cat_model.predict_proba(X_test)[:,1]
auc_cat = roc_auc_score(y_test, y_pred_catboost)
print(f"CatBoost ROC AUC : {auc_cat:.3f}")


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=3]="A32": Cannot convert 'A32' to float

# estimation d'un modèle de Forêt aléatoire

In [29]:
# --- 4. Forêt Aléatoire ---
rf_model = RandomForestClassifier(n_estimators=500, max_depth=None, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict_proba(X_test)[:,1]
auc_rf = roc_auc_score(y_test, y_pred_rf)
print(f"Random Forest ROC AUC : {auc_rf:.3f}")

ValueError: could not convert string to float: 'A32'

# Propre

In [ ]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import statsmodels.api as sm
import matplotlib.pyplot as plt

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'